In [1]:
import pandas as pd
import json
import numpy as np
from tabulate import tabulate
from helpers_coordiantes import convert_coords

In [3]:
segments_file = '/Users/aruscha/Documents/cut-parkplatz-neu/datasets/complete/2021_12_22_sdegdw_f66_d_StrassenSegmente.geojson'
area_file_merged = '/Users/aruscha/Documents/cut-parkplatz-neu/datasets/complete/leipzig_verkehrsflaechen_breite_gdi_merged_20230504.json'

## merge breite into verkehrsflaechen json

In [ ]:
# --- specify file paths -----#
area_file = '/Users/aruscha/Documents/cut-parkplatz-neu/datasets/complete/leipzig_verkehrsflaechen.json'
area_file_with_breite = '/Users/aruscha/Documents/cut-parkplatz-neu/datasets/complete/leipzig_verkehrsflaechen_breite.json'
# --------------#   
# Load the JSON file into a Python object
with open(area_file_with_breite, 'r') as f:
    breite_data = json.load(f)

#Extract the "features" array from the JSON data
breite_features = breite_data['features']

#Normalize the "properties" object for each feature and create a Pandas dataframe
breite_df = pd.concat([pd.json_normalize(item) for item in breite_features], axis=0)
print(breite_df.columns)

with open(area_file, 'rb') as f1:
    area_data = json.load(f1)
area_features = area_data['features']

#Normalize the "properties" object for each feature and create a Pandas dataframe
area_df = pd.concat([pd.json_normalize(item) for item in area_features], axis=0)
area_df
area_df['properties.median_bre'] = np.nan
print(area_df.columns)

In [40]:
feature_list = []
for index, row in area_df.iterrows():
    obj_id = row['properties.objectid']

    try:
        median_bre = breite_df.loc[breite_df['properties.objectid']==obj_id, 'properties.median_bre'].values[0]
    except IndexError:
        median_bre = -1
  
    #area_df.loc[area_df['properties.objectid'] == obj_id,'properties.median_bre'] = median_bre
    area_df.at[index, 'properties.median_bre'] = median_bre
    properties = dict()
    for col in area_df.columns:
        if "properties" in col:
            key = col.split(".")[1]
            properties[key] = row[col]

    feature = {"type" : "Feature",
               "geometry" : {
                   "type" : row['geometry.type'],
				    "coordinates" : row['geometry.coordinates']
                    },
               "properties" :  properties
               }
    feature_list.append(feature)



json_res = {
	"type" : "FeatureCollection",
	"name" : "leipzig_verkehrsflaechen",
	"features" : feature_list
}
# Write the normalized DataFrame back to a JSON file
with open("test.json", 'w', encoding="utf-8") as f2:
    json.dump(json_res, f2, ensure_ascii=False)


## read data to dfs and write back to json

In [5]:
# extract data from strassen_segmente_testgebiet.json - file the city sent us
def extract_segment_data(segments_json):
    street_items = []
    for feature in segments_json['features']:
        geom_type = feature['geometry']['type']
        coordinates = feature['geometry']['coordinates']
        #coordinates = [(item[1], item[0]) for item in coordinates]
        str = feature['properties']['str']
        segm_gid = feature['properties']['segm_gid']
        objectid = feature['properties']['objectid']
        von_str = feature['properties']['von_str']
        bis_str = feature['properties']['bis_str']
        str_nr = feature['properties']['str_nr']
        kat_sstrgs = feature['properties']['kat_sstrgs']
        sstrgsname = feature['properties']['sstrgsname']
        laenge_m = feature['properties']['laenge_m']
        street_items.append({'str_nr': str_nr, 'str': str,'objectid': objectid, 'segm_gid': segm_gid, 'kat_sstrgs': kat_sstrgs, 'sstrgsname': sstrgsname, 'von_str': von_str, 'bis_str': bis_str, 'laenge_m': laenge_m, 'geom_type': geom_type, 'coordinates': coordinates})
    
    street_df = pd.DataFrame.from_records(street_items)
    return street_df


# write segments data from pandas df back to json
def write_segments_to_Json(segments_df):
    feature_list = []
    for index, row in segments_df.iterrows():
        
        properties = {'str_nr': row['str_nr'], 'str': row['str'],'objectid': row['objectid'], 'segm_gid': row['segm_gid'], 'kat_sstrgs': row['kat_sstrgs'], 'sstrgsname': row['sstrgsname'], 'von_str': row['von_str'], 'bis_str': row['bis_str'], 'laenge_m': row['laenge_m']}
     
        feature = {"type" : "Feature",
                "geometry" : {
                    "type" : row['geom_type'],
                     "coordinates" : row['coordinates']
                        },
                "properties" :  properties
                }
        feature_list.append(feature)
    return feature_list

In [52]:
with open(segments_file, 'rb') as f1:
    segments_json = json.load(f1)

    #extract segments
segments_df = extract_segment_data(segments_json)
print(tabulate(segments_df.head(), headers='keys', tablefmt='psql'))
print(segments_df['street_type'].value_counts())

+----+-------------+--------------------+-------------+------------+-------+---------------+---------------+-------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+
|    |   street_nr | str                |   object_id |   segm_gid | cat   | street_type   | from_str      | to_street         | coordinates                                                                                                                                                                                       |   length |
|----+-------------+--------------------+-------------+------------+-------+---------------+---------------+-------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----

In [6]:
def extract_area_data(segments_json):
    street_items = []
    for feature in segments_json['features']:
        geom_type = feature['geometry']['type']
        coordinates = feature['geometry']['coordinates'][0]
        #print(coords)
        if geom_type == "MultiPolygon":
            for coord_list in coordinates:
                coord_list = [(item[1], item[0]) for item in coord_list]
        else:
            coordinates = [(item[1], item[0]) for item in coordinates]
        str = feature['properties']['str']
        objectid = feature['properties']['objectid']
        stsa_text = feature['properties']['stsa_text']
        segm_gid = feature['properties']['segm_gid']
        laenge = feature['properties']['laenge']
        median_breite = feature['properties']['median_breite']

        street_items.append({'str': str,'objectid': objectid, 'stsa_text': stsa_text, 'segm_gid': segm_gid, 'laenge': laenge, 'median_breite': median_breite, 'geom_type': geom_type, 'coordinates': coordinates})
    
    street_df = pd.DataFrame.from_records(street_items)
    return street_df


def write_area_to_Json(area_df):
    feature_list = []
    for index, row in area_df.iterrows():
        
        properties = {'str': row['str'],'objectid': row['objectid'], 'stsa_text': row['stsa_text'], 'segm_gid': row['segm_gid'],'median_breite': row['median_breite'], 'laenge': row['laenge']}
     
        feature = {"type" : "Feature",
                "geometry" : {
                    "type" : row['geom_type'],
                     "coordinates" : row['coordinates']
                        },
                "properties" :  properties
                }
        feature_list.append(feature)
    return feature_list

In [5]:
area_file = '/Users/aruscha/Documents/cut-parkplatz-neu/datasets/complete/leipzig_verkehrsflaechen_breite_gdi_merged_20230504.json'
    
with open(area_file, 'rb') as f2:
    area_json = json.load(f2)

    #extract segments
area_df = extract_area_data(area_json)
print(area_df['stsa_text'].value_counts())



area_df = area_df.loc[area_df['stsa_text'].str.contains("Fahrbahn")]
print(len(area_df))
print(tabulate(area_df.head(), headers='keys', tablefmt='psql'))

Gehweg                                                                        17119
Grünfläche                                                                    15205
Fahrbahn (dazu gehören Parkbuchten, Stellflächen, Radstreifen, Busbuchten)    13015
Seitenstreifen, Randstreifen, Bankett                                          4240
Seitentrennstreifen                                                            1793
Straßengraben, Straßenmulde                                                    1371
Querungshilfe                                                                  1212
Radweg                                                                         1145
Straßenbündiger Bahnkörper                                                     1128
Böschung                                                                       1112
Geh- und Radweg                                                                 686
Verkehrsteiler                                                              

# PREprocessing

### Edit segmente file: remove Parkplatz

In [7]:
with open(segments_file, 'rb') as f1:
    segments_json = json.load(f1)
segments_df = extract_segment_data(segments_json)
segments_df = segments_df.loc[~segments_df['sstrgsname'].str.contains("Parkplatz")]

print(tabulate(segments_df.head(), headers='keys', tablefmt='psql'))

feature_list = write_segments_to_Json(segments_df)

json_res = {
	"type" : "FeatureCollection",
	"name" : "leipzig_verkehrsflaechen",
	"features" : feature_list
}
# Write the normalized DataFrame back to a JSON file
with open("/Users/aruscha/Documents/cut-parkplatz-neu/datasets/complete/2021_12_22_sdegdw_f66_d_StrassenSegmente_CLEANED.json", 'w', encoding="utf-8") as f2:
    json.dump(json_res, f2, ensure_ascii=False)

+----+----------+--------------------+------------+------------+--------------+--------------+---------------+-------------------+------------+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|    |   str_nr | str                |   objectid |   segm_gid | kat_sstrgs   | sstrgsname   | von_str       | bis_str           |   laenge_m | geom_type   | coordinates                                                                                                                                                                                       |
|----+----------+--------------------+------------+------------+--------------+--------------+---------------+-------------------+------------+-------------+---------------------------------------------------------------------------------------------------------------------------------------

## edit area file: keep only fahrbahn

In [25]:
with open(area_file_merged, "rb") as f2:
    area_json = json.load(f2)
area_df = extract_area_data(area_json)
area_df= area_df.loc[area_df['stsa_text'].str.contains("Fahrbahn")]
print(area_df.head())

feature_list = write_area_to_Json(area_df)
json_area_new = {
	"type" : "FeatureCollection",
	"name" : "vta_verkehrsflaechen_breite_20230504",
    "features": feature_list }

# Write the normalized DataFrame back to a JSON file
with open("/Users/aruscha/Documents/cut-parkplatz-neu/datasets/complete/leipzig_verkehrsflaechen_breite_gdi_merged_20230504_CLEANED.json", 'w', encoding="utf-8") as f2:
    json.dump(json_area_new, f2, ensure_ascii=False)

                   str  objectid  \
5   Delitzscher Straße   1796090   
9   Delitzscher Straße   1796094   
15  Delitzscher Straße   1796099   
16  Delitzscher Straße   1796100   
22  Eutritzscher Markt   1796106   

                                            stsa_text  segm_gid  laenge  \
5   Fahrbahn (dazu gehören Parkbuchten, Stellfläch...   24423.0     117   
9   Fahrbahn (dazu gehören Parkbuchten, Stellfläch...   24436.0      78   
15  Fahrbahn (dazu gehören Parkbuchten, Stellfläch...   24437.0     190   
16  Fahrbahn (dazu gehören Parkbuchten, Stellfläch...   24437.0     190   
22  Fahrbahn (dazu gehören Parkbuchten, Stellfläch...   24448.0      41   

    median_breite geom_type                                        coordinates  
5            5.95   Polygon  [(51.3737362026, 12.3856210525), (51.373649159...  
9            3.48   Polygon  [(51.3744352496, 12.3855664464), (51.374369288...  
15           7.29   Polygon  [(51.3761427116, 12.3854193873), (51.376010757...  
16      

## convert coordinates of ortsteile file from 4326 to 25833 for postgis

In [6]:
ot_file = '/Users/aruscha/Documents/cut-parkplatz-neu/datasets/complete/leipzig_ortsteile.json'
    
with open(ot_file, 'rb') as f3:
    ot_json = json.load(f3)
    for feature in ot_json['features']:
        coordinates = feature['geometry']['coordinates']
        conv_coords = [[list(convert_coords("EPSG:4326", "EPSG:25833", pt[1], pt[0])) for pt in coordinates[0]]]
        feature['geometry']['coordinates'] = conv_coords
        
with open("/Users/aruscha/Documents/cut-parkplatz-neu/datasets/complete/leipzig_ortsteile_CLEANED.json", 'w', encoding="utf-8") as f4:
    json.dump(ot_json, f4, ensure_ascii=False)